In [1]:
import pandas as pd
import numpy as np
import re
import keras
from keras.models import Sequential
from keras.layers import Activation,LSTM,Dense
from keras.optimizers import Adam
import random

/Users/tanvirniaz/Documents/Python/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
lyricstring = open('alllyrics.txt').read() #opens .txt file with lyrics scrape from Genius
lyricstring2 = re.sub('[^a-zA-Z0-9Z*\n\.]', ' ', lyricstring) #cleans text data
lyriclist=lyricstring2.split('**') #splits into list with placeholder marker **
print (lyriclist) 

['  0 to 100   The Catch Up     Produced by Boi 1da  Frank Dukes  Noah  40  Shebib    Nineteen85  n n Part I  0 to 100  n n Verse 1  nFuck bein   on some chill shit nWe go 0 to 100  nigga  real quick nThey be on that rap to pay the bill shit nAnd I don  t feel that shit  not even a little bit nOh Lord  know yourself  know your worth  nigga nMy actions been louder than my words  nigga nHow you so high  but still so down to Earth  nigga  nIf niggas wanna do it  we can do it on they turf  nigga nOh Lord  I  m the rookie and the vet nShoutout to the bitches out here holdin   down the set nAll up in my phone  lookin   at pictures from the other night nShe gon   be upset if she keep scrollin   to the left  dawg nShe gon   see some shit that she don  t wanna see nShe ain  t ready for it nIf I ain  t the greatest then I  m headed for it nThat mean I  m way up  way up  nYeah  the 6 ain  t friendly but that  s where I lay up nThis shit a mothafuckin   lay up nI been Steph Curry with the shot nBe

In [3]:
alltxt='' 
for i in range (len(lyriclist)):
    alltxt+=lyriclist[i] #saves all lyrics combined into one string alltxt
print (alltxt)

  0 to 100   The Catch Up     Produced by Boi 1da  Frank Dukes  Noah  40  Shebib    Nineteen85  n n Part I  0 to 100  n n Verse 1  nFuck bein   on some chill shit nWe go 0 to 100  nigga  real quick nThey be on that rap to pay the bill shit nAnd I don  t feel that shit  not even a little bit nOh Lord  know yourself  know your worth  nigga nMy actions been louder than my words  nigga nHow you so high  but still so down to Earth  nigga  nIf niggas wanna do it  we can do it on they turf  nigga nOh Lord  I  m the rookie and the vet nShoutout to the bitches out here holdin   down the set nAll up in my phone  lookin   at pictures from the other night nShe gon   be upset if she keep scrollin   to the left  dawg nShe gon   see some shit that she don  t wanna see nShe ain  t ready for it nIf I ain  t the greatest then I  m headed for it nThat mean I  m way up  way up  nYeah  the 6 ain  t friendly but that  s where I lay up nThis shit a mothafuckin   lay up nI been Steph Curry with the shot nBeen

In [4]:
vocab=list(set(alltxt)) #list of all unqiue characters in txt- list comprehension
char_ix={c:i for i,c in enumerate(vocab)} #dictionary for vocab- dictionary comprehension
ix_char={i:c for i,c in enumerate(vocab)} #dictionary for vocab- dictionary comprehension
vocabsize=len(vocab) #total length of vocab set

In [5]:
maxlen=40 #max length of artbitrary sentence length
sentences=[] #placeholder for sentences
nextchar=[] #placeholder for next character
for i in range(len(alltxt)-maxlen-1): #iterate through alltxt
    sentences.append(alltxt[i:i+maxlen]) #list of all length 40 sentences
    nextchar.append(alltxt[i+maxlen]) #list of all next characters (every 40)

In [6]:
#initializing x,y arrays for modelling
x=np.zeros((len(sentences),maxlen,vocabsize)) #x is array of 3D shape a*b*c
y=np.zeros((len(sentences),vocabsize)) #y is array of 2D shape a*c

In [7]:
#populating x,y arrays using dictionary for vocab
for ix in range(len(sentences)): #iterate over range of total sentences
    y[ix,char_ix[nextchar[ix]]]=1 #y array index [ix,where char_ix=nextchar[ix]] becomes true (iterate through x-axis)
    for iy in range(maxlen): #iterate over range of sentence length
        x[ix,iy,char_ix[sentences[ix][iy]]]=1 #x array index [ix,iy,char_ix=sentences[ix][iy]] becomes true (iterate through x and y-axis)

In [8]:
#modelling using a RNN composed of LTSM and regular layers
model=Sequential()
model.add(LSTM(349,input_shape=(maxlen,vocabsize))) #batch size: 10 songs, timesteps: 40 (maxlen), input_dim/features: len(vocab)
model.add(Dense(vocabsize)) #add regular densely connected NN layers (len(vocab))
model.add(Activation('softmax')) #final softmax activation layer
model.summary()
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy') #configure model for training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 349)               579340    
_________________________________________________________________
dense_1 (Dense)              (None, 65)                22750     
_________________________________________________________________
activation_1 (Activation)    (None, 65)                0         
Total params: 602,090
Trainable params: 602,090
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(x,y,epochs=5,batch_size=349) #fitting model to x,y data

Epoch 1/5
826574/826574 [==============================] - 3253s 4ms/step - loss: 1.7959
Epoch 2/5
826574/826574 [==============================] - 3082s 4ms/step - loss: 1.4730
Epoch 3/5
826574/826574 [==============================] - 3011s 4ms/step - loss: 1.5368
Epoch 4/5
826574/826574 [==============================] - 3123s 4ms/step - loss: 1.6771
Epoch 5/5
826574/826574 [==============================] - 3029s 4ms/step - loss: 1.6514


In [17]:
#setting a random starting index and sentence for predicting
generated ='' #intializing generated string
start_index=random.randint(0,len(alltxt)-maxlen-1) #random start index between 0 and last next char value over len(txt)
sent=alltxt[start_index:start_index+maxlen] #sentence from random start index
generated+=sent #add sentence value to generated

#converting generated string to array of corresponding vocab dictionary values
for i in range(1900): #for range of total song lyrics length (1900 elements)
    x_sample=generated[i:i+maxlen] #save generated characters to array x_sample
    samp=np.zeros((1,maxlen,vocabsize)) #initialize samp array of (1*b*c)
    for j in range(maxlen): #iterate over y-axis
        samp[0,j,char_ix[x_sample[j]]]=1 #converting string array to index array; values are true for x_sample values iterated along y-axis (sentence length)
    probs=model.predict(samp) #based on sample input of 1*b*c, predicts other x-values
    probs=np.reshape(probs,probs.shape[1]) #reshape prediction
    ix=np.random.choice(range(vocabsize),p=probs.ravel()) #returns random element (word index) from 1-D array or prediction
    generated+=ix_char[ix] #match char values to given random element (word)

In [13]:
print (generated) #first generated lyric

ro  PARTYNEXTDOOR  nOh  it  s your fuckin nGassion  nWorking  I don  t want to colla as crazy they des9roon obped nKunds head the poitin   dollary  I m sec out nToo many selfed and here yo9  he good nWell worls and the fuckin   nGirl when he different tto brokes are darked me nI get a baby go nBoung nI get it  I  m s9 Cleep me the of when I9parp nThe9 here I don t want me but gi9s fest nLet we re9tins  never lil it  it s down Qust planed signs in Leath dance hard to pick me nLose the poQ different thouglan nI I  I 9I  I m ready for you for your bo9 90 9309 n9.5 I  m   they don t nvell   em Driz nAn99 9 fays what I m down   nYou never got bout  the youn9 nThe milted actire nBam friend cash my bout my bla Sgenia antor of Athinue get me nThyn   growly  the begreis  fin that nI ve done but I just tell them pood nigh9  lookin  nBu9p skin  All I have Rose  Who s right  drop  nLil you9f can t know ya   uh  but you wa9t and you not from nCan t eat with you to peace hop  yo bees nI feelin  spop

In [16]:
print (generated) #second generated lyric

 America nPart of my heritage  pardon my other niggas could do nUntil that  s yor nMy thing  tell you are s cordagrwen spain nAnd they penny  spend to find you and an tonisted nA druils with me better find the mille nWhat you know not call me all the need my city wetten here I can gug We the ends nYou know I got a 9216 froup  n n Chorus  Young Ma9 Hopodyon*  nYou niggas keeping my intor as this provements  I sturd the Kester she just spitce  crap9 before the can rejiereched around a tiggy nCash I m talking you down like the less nI do9  t niggas want it  manthe ruit you9 people tryna let   em got no baped  They dad nervind of Week reeas of a game  it givl  Which   nVount nAnd flowndetion call tell something  gangstas  chased in listy nAnd big  true  baby I do now nI m done with your down in the9gated Paa9ance   bout to met nNo tell me down nI  m done that I got the riggrs nBlackst9d if you smade9 is long for you  oh hoh nChapi nFuck you a write to fict it nAnd Hulf my one  I wonder nNo

In [18]:
print (generated) #third generated lyric

ma go hard  and I  ma go hard nAnd I  ma been a grown nI9  yaam  just looki9   nAsk what am han9 to please to back  ayy  you don  t miss my trove n   Rove 8016 12 n  Oliver n nEpisode 50 n nEpiso9 is cups to charys. It don t9eppuse good up evolicres from9 9 list n n Hook  nWatter  child thought pedpriat  and much no from  nI  m runging good growin  bitsh else and9Ad nI I  I m ready try long nAnd showed and 929ch9thant anythin  I m up now  I took a thusy nGot no reall9ous alright9 was nigga  straight  nHolda down fuck   n n Verse 1  DJAYEE  nI just wanna know  this is just up  hah 9  you   bout back in the perfect  emotion nPhett when I have sur our troom nThis that so I kno9 show it nYou know the mamal girl 99 soor nAig to over Fad  coside a like  boy nI  m appeorful  stant sket again three e9 is dookly fity I don  t lie deal nAnd nYou over sings w9tched   over so keep to predie you so right it up by my city nEvery shop is sposed out crap  ain t buzzman nOr and we had a sucker in a mot